In [1]:
# !pip install --upgrade google-cloud-speech

In [3]:
# !pip install rasa_nlu

In [8]:
# !pip install wit

In [70]:
# !pip install --upgrade google.cloud

  Using cached https://files.pythonhosted.org/packages/ba/b1/7c54d1950e7808df06642274e677dbcedba57f75307adf2e5ad8d39e5e0e/google_cloud-0.34.0-py2.py3-none-any.whl


In [30]:
# !pip install --upgrade google-cloud-language

In [60]:
# !pip install --upgrade setuptools pip

  Found existing installation: setuptools 40.6.3
    Uninstalling setuptools-40.6.3:
      Successfully uninstalled setuptools-40.6.3
  Found existing installation: pip 18.1
    Uninstalling pip-18.1:
      Successfully uninstalled pip-18.1


Could not install packages due to an EnvironmentError: [WinError 5] Access is denied: 'C:\\Users\\risha\\AppData\\Local\\Temp\\pip-uninstall-f88009si\\users\\risha\\anaconda3\\scripts\\pip.exe'
Consider using the `--user` option or check the permissions.



# Libraries

In [344]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import math
import seaborn as sns
from sklearn.utils import shuffle
import sklearn.linear_model as Lm
from sklearn.linear_model import LinearRegression as Lr
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import Perceptron
import requests
import io
import os
from google.cloud import speech
from google.cloud.speech import enums as enums1
from google.cloud.speech import types
from rasa_nlu.model import Metadata, Interpreter
from wit import Wit
import json
from google.cloud import language_v1
from google.cloud.language_v1 import enums
from nltk import tokenize
from nltk.stem import PorterStemmer

In [320]:
te = getTextFromAudio("Demo_5.flac")

In [321]:
te

'Can I order a burger and fries I would like to order one large pizza and one soda, I would like to get a coffee to all right, I would like to order one cheeseburger and 5 price and Treecko and 7 Sandwich one go to.'

In [260]:
te1 = te.replace(",",".")
te1 = te1.split(".")
te1[2]

' I would like to order one cheeseburger and 5 price and Treecko and 7 Sandwich one go to'

In [317]:
def getTextFromAudio(filename):
    client = speech.SpeechClient()

    file_name = os.path.join('Audio Files',filename)

    with io.open(file_name, 'rb') as audio_file:
        content = audio_file.read()
        audio = types.RecognitionAudio(content=content)

    config = types.RecognitionConfig(
        enable_automatic_punctuation = True,
        encoding=enums1.RecognitionConfig.AudioEncoding.FLAC,
        sample_rate_hertz=16000,
        language_code='en-US')

    response = client.recognize(config, audio)

    for result in response.results:
        return result.alternatives[0].transcript

In [151]:
def analyze_entities(text_content):
    client = language_v1.LanguageServiceClient()
    type_ = enums.Document.Type.PLAIN_TEXT
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}
    encoding_type = enums.EncodingType.UTF8
    response = client.analyze_entities(document, encoding_type=encoding_type)
    words = text_content.split(" ")
    d = {}
    for i in words:
        d[i] = -1
        
    for entity in response.entities:
        for mention in entity.mentions:
            if (enums.Entity.Type(entity.type).name != "NUMBER"):
                d[entity.name] = enums.EntityMention.Type(mention.type).name
            else:
                d[entity.name] = entity.metadata['value']
    return d


In [338]:
b = MakeMenu(Database)
b.getIntent("Add three Quesadilla")

Add three Quesadilla


'Add'

In [152]:
analyze_entities("hi I would like to order one burger and 5 fries and 3 coke and 7 zip")

{'hi': -1,
 'I': -1,
 'would': -1,
 'like': -1,
 'to': -1,
 'order': -1,
 'one': '1',
 'burger': 'COMMON',
 'and': -1,
 '5': '5',
 'fries': 'COMMON',
 '3': '3',
 'coke': 'COMMON',
 '7': '7',
 'zip': 'COMMON'}

In [147]:
from google.cloud import language_v1
from google.cloud.language_v1 import enums


def sample_analyze_entities(text_content):
    """
    Analyzing Entities in a String

    Args:
      text_content The text content to analyze
    """

    client = language_v1.LanguageServiceClient()

    # text_content = 'California is a state.'

    # Available types: PLAIN_TEXT, HTML
    type_ = enums.Document.Type.PLAIN_TEXT

    # Optional. If not specified, the language is automatically detected.
    # For list of supported languages:
    # https://cloud.google.com/natural-language/docs/languages
    language = "en"
    document = {"content": text_content, "type": type_, "language": language}

    # Available values: NONE, UTF8, UTF16, UTF32
    encoding_type = enums.EncodingType.UTF8

    response = client.analyze_entities(document, encoding_type=encoding_type)
    # Loop through entitites returned from the API
    for entity in response.entities:
        print(u"Representative name for the entity: {}".format(entity.name))
        # Get entity type, e.g. PERSON, LOCATION, ADDRESS, NUMBER, et al
        print(u"Entity type: {}".format(enums.Entity.Type(entity.type).name))
        # Get the salience score associated with the entity in the [0, 1.0] range
        print(u"Salience score: {}".format(entity.salience))
        # Loop over the metadata associated with entity. For many known entities,
        # the metadata is a Wikipedia URL (wikipedia_url) and Knowledge Graph MID (mid).
        # Some entity types may have additional metadata, e.g. ADDRESS entities
        # may have metadata for the address street_name, postal_code, et al.
        if len(entity.metadata) > 0:
            print(entity.metadata['value'])

        # Loop over the mentions of this entity in the input document.
        # The API currently supports proper noun mentions.
        for mention in entity.mentions:
            print(u"Mention text: {}".format(mention.text.content))
            # Get the mention type, e.g. PROPER for proper noun
            print(
                u"Mention type: {}".format(enums.EntityMention.Type(mention.type).name)
            )

    # Get the language of the text, which will be the same as
    # the language specified in the request or, if not specified,
    # the automatically-detected language.
    print(u"Language of the text: {}".format(response.language))


In [357]:
Database = {
    'coffee' : 5,
    'tea' : 5,
    'burger' : 10,
    'sandwich' : 5,
    'pizza' : 10,
    'soda' : 3,
    'fries' : 4.5,
    'coke' : 3,
    'burritos' : 10,
    'nachos' : 5,
    'quesadilla' : 6.5
}

In [354]:
class MakeMenu:
    
    def __init__(self, database):
        self.menu = {}
        self.WitClient = Wit("QIPH5735KKLG3SRKLW6ULLPAA3FCYYIS")
        self.database = database
        self.porter = PorterStemmer()
    
    def generateMenu(self, sentences):
        mainMenu = {}
        for i in sentences:
            if (len(i) < 2):
                continue
            i = i.lower()
            intent = self.getIntent(i)
            oneMenu = self.generateOneMenu(i)
            print(mainMenu)
            if (intent == 'Add'):
                for i in oneMenu:
                    if (i in mainMenu):
                        mainMenu[i] += oneMenu[i]
                    else:
                        mainMenu[i] = oneMenu[i]
            elif (intent == 'Delete'):
                for i in oneMenu:
                    if (i in mainMenu):
                        mainMenu[i] -= oneMenu[i]
            
        receipt = []
        for i in mainMenu:
            if i not in self.database:
                continue
            temp = []
            temp.append(i)
            temp.append(mainMenu[i])
            temp.append(self.database[i])
            receipt.append(temp)
        FinalMenu = pd.DataFrame(receipt)
        FinalMenu.columns = ['Item', 'Quantity', 'Price']
        return FinalMenu
                
        
    def generateOneMenu(self, sentence):
        oneMenu = {}
        words, d = self.analyze_entities(sentence)
        for i in range(len(words)):
            if d[words[i]] == 'COMMON':
                quant = 1
                if (i != 0 and (str(d[words[i-1]]).isdigit()) and d[words[i-1]] != -1):
                    quant = d[words[i-1]]
                oneMenu[words[i]] = quant
        print(d)
        return oneMenu
        
        
    def getIntent(self, sentence):
        resp = self.WitClient.message(sentence)   
        print(sentence)
        return resp['entities']['intent'][0]['value']
    
    def analyze_entities(self, sentence):
        client = language_v1.LanguageServiceClient()
        type_ = enums.Document.Type.PLAIN_TEXT
        language = "en"
        sentence = sentence.split(" ")
        for i in range(len(sentence)):
            sentence[i] = sentence[i].lower()
        sentence = " ".join(sentence)
        document = {"content": sentence, "type": type_, "language": language}
        encoding_type = enums.EncodingType.UTF8
        response = client.analyze_entities(document, encoding_type=encoding_type)
        words = sentence.split(" ")
        for i in range(len(words)):
            words[i] = words[i].lower()
        d = {}
        for i in words:
            d[i] = -1

        for entity in response.entities:
            for mention in entity.mentions:
                if (enums.Entity.Type(entity.type).name != "NUMBER"):
                    d[entity.name.lower()] = enums.EntityMention.Type(mention.type).name
                else:
                    d[entity.name] = int(entity.metadata['value'])
        return words, d

        

In [309]:
senten = []
senten.append('Can i order five Burger and fries')
senten.append('I would like to order one large pizza and one soda')
senten.append('Please remove the burger from my order')
senten.append('I Would like to get a coffee too')
senten.append('hi I would like to order one cheese burger and 5 fries and 3 coke and 7 sandwich')
senten.append('Remove one coke too')

In [322]:
a = MakeMenu(Database)
menu = a.generateMenu(senten)
menu

{}
{'burger': 5, 'fries': 1}
{'burger': 5, 'fries': 1, 'pizza': 1, 'soda': 1}
{'burger': 4, 'fries': 1, 'pizza': 1, 'soda': 1}
{'burger': 4, 'fries': 1, 'pizza': 1, 'soda': 1, 'coffee': 1}
{'burger': 4, 'fries': 6, 'pizza': 1, 'soda': 1, 'coffee': 1, 'coke': 3, 'sandwich': 7}


,Item,Quantity,Price
0,burger,4,10.0
1,fries,6,4.5
2,pizza,1,10.0
3,soda,1,3.0
4,coffee,1,5.0
5,coke,2,3.0
6,sandwich,7,5.0


In [323]:
print(menu)
print('Total Number of Items:', menu['Quantity'].sum())
print('Total Price:', menu['Price'].sum())

       Item  Quantity  Price
0    burger         4   10.0
1     fries         6    4.5
2     pizza         1   10.0
3      soda         1    3.0
4    coffee         1    5.0
5      coke         2    3.0
6  sandwich         7    5.0
Total Number of Items: 22
Total Price: 40.5


In [355]:
senten2 = []
senten2.append('Can i get two burritos and nachos on side')
senten2.append('Add 3 Quesadilla')
senten2.append('Remove 1 burritos')
senten2.append('Can i also get three coke')

In [358]:
b = MakeMenu(Database)
menu = b.generateMenu(senten2)
menu

can i get two burritos and nachos on side
{'can': -1, 'i': -1, 'get': -1, 'two': 2, 'burritos': 'COMMON', 'and': -1, 'nachos': 'COMMON', 'on': -1, 'side': 'COMMON'}
{}
add 3 quesadilla
{'add': -1, '3': 3, 'quesadilla': 'COMMON'}
{'burritos': 2, 'nachos': 1, 'side': 1}
remove 1 burritos
{'remove': -1, '1': 1, 'burritos': 'COMMON'}
{'burritos': 2, 'nachos': 1, 'side': 1, 'quesadilla': 3}
can i also get three coke
{'can': -1, 'i': -1, 'also': -1, 'get': -1, 'three': 3, 'coke': 'COMMON'}
{'burritos': 1, 'nachos': 1, 'side': 1, 'quesadilla': 3}


,Item,Quantity,Price
0,burritos,1,10.0
1,nachos,1,5.0
2,quesadilla,3,6.5
3,coke,3,3.0
